In [22]:
import pandas as pd
import numpy as np
import nibabel as nib
import datetime
import os
import random
from nilearn.input_data import NiftiMasker

In [18]:
# Parameters
toy = False
run_all = True
path = '/home/ubuntu/fmriprep/'
output_dir = '../output/'

# input_dir = '/home/ubuntu/brain_age/drsulliv3/'
input_dir = '/Users/danielmlow/Dropbox/neurohack/brain_age/data/CNP_clinical/func/' #Daniel's PC


n_jobs = 20 #amount of cores
cv=4
description = 'CNP_clinical_T1_age_'
log_file = description+datetime.datetime.now().strftime("%y-%m-%d-%H-%M-%S")

In [33]:
log_file

'CNP_func_gridsearch_age_18-08-09-19-17-53'

In [8]:
population = 'adhd'
parcellation = '444'
loaded = np.load(input_dir+'CNP_func_gridsearch_age_18-08-10-02-50-17CNP_func'+population+'_'+parcellation+'_train_set.npz')
y_adhd, subj_adhd = loaded['b'], loaded['c']

population = 'bipolar'
parcellation = '444'
loaded = np.load(input_dir+'CNP_func_gridsearch_age_18-08-10-02-50-17CNP_func'+population+'_'+parcellation+'_train_set.npz')
y_bipolar, subj_bipolar = loaded['b'], loaded['c']

population = 'bipolar'
parcellation = '444'
loaded = np.load(input_dir+'CNP_func_gridsearch_age_18-08-10-02-50-17CNP_func'+population+'_'+parcellation+'_train_set.npz')
y_bipolar, subj_bipolar = loaded['b'], loaded['c']


In [19]:
# Load volumes for each subject into a dictionary    


def filenames(subj):
    filenames = []
    for i in range(len(subj)):
        filenames.append(path+'sub-'+subj[i]+'/anat/sub-'+subj[i]+'_T1w_space-MNI152NLIN2009cAsym_preprop.nii.gz')
    return filenames

filenames_adhd  = filenames(subj_adhd)
filenames_schz  = filenames(subj_schz)
filenames_bipolar  = filenames(subj_bipolar)


In [15]:
from nilearn.input_data import NiftiLabelsMasker
from nilearn.connectome import ConnectivityMeasure, sym_matrix_to_vec

def t1_2_matrix(name, filenames, y, subj):    
    path_and_filenames = [path+n for n in filenames]
    nifti_masker = NiftiMasker(
        standardize=False,
        smoothing_fwhm=2, mask_strategy='epi',
        memory='nilearn_cache')  # cache options
    gm_maps_masked = nifti_masker.fit_transform(filenames)
    print("%d samples, %d features" % (n_samples, n_features))
    #   Split train test sets and save
    split = len(gm_maps_masked)/2.
    X_train = gm_maps_masked[:split]
    X_test = gm_maps_masked[split:]
    y_train = y[:split]
    y_test = y[split:]
    random_subj_train = subj[:split]
    random_subj_test = subj[split:]
    np.savez_compressed(output_dir+'CNP_T1_'+name+'_train_set',a=X_train, b=y_train, c=random_subj_train)
    np.savez_compressed(output_dir+'CNP_T1_'+name+'_test_set',a=X_test, b=y_test, c=random_subj_test)
    return gm_maps_masked


gm_maps_masked = t1_2_matrix('adhd', filenames_adhd, y_adhd, subj_adhd)
gm_maps_masked = t1_2_matrix('schz', filenames_schz, y_schz, subj_schz)
gm_maps_masked = t1_2_matrix('bipolar', filenames_bipolar, y_bipolar, subj_bipolar)